In [22]:

import numpy as np
import torch
import os
from sklearn.neighbors import KDTree
from torch_geometric.data import Data as GraphData
import pymysql
import yaml
from tqdm import tqdm
import random
import types

In [23]:
def connect():
    with open('database.yaml', 'r') as file:
        database_info = yaml.safe_load(file)
    mysql_info = database_info['mysql']
    host = mysql_info['host']
    user = mysql_info['user']
    passwd = mysql_info['passwd']
    return pymysql.connect(host=host, user=user, passwd=passwd, db='edge_task_scheduling', charset='utf8')

In [29]:
arg =  types.SimpleNamespace()

arg.dataset = "alibaba_cluster_trace"
arg.task_n = 100
arg.epoch = 10
arg.batch = 5
self = types.SimpleNamespace()

In [33]:
def get_data_instance(self, arg):
    query = 'SELECT MAX(id) FROM {}'.format(self.dataset)
    self.cursor.execute(query)
    max_id = self.cursor.fetchone()[0] - arg.task_n
    start_id = random.randint(0, max_id)
    # 从一个随机id开始抽取数据
    query = 'SELECT * FROM {} WHERE id > {} LIMIT {}'.format(self.dataset, start_id, arg.task_n)
    self.cursor.execute(query)
    # [id, cpu,io,bandwidth,ram,timestamp,duration,latitude,longitude]
    task_list = self.cursor.fetchall()
    task_list = np.array(task_list)[:, 1:-2]
    task_list[:, 4] = abs(task_list[:, 4] - task_list[-1][4])
    return torch.tensor(task_list, dtype=torch.float32).unsqueeze(0)

In [54]:
self.db = connect()
self.cursor =  self.db.cursor()
self.dataset = arg.dataset
self.file_lines = []
for _ in tqdm(range(arg.batch * arg.epoch)):
  instance = get_data_instance(self,arg)
  instance = instance.squeeze(0).tolist()
  self.file_lines.append(instance)
print((self.file_lines[0][0]))
print(f'Loaded "{self.dataset}"')

100%|██████████| 50/50 [00:00<00:00, 330.59it/s]

[0.11999999731779099, 0.08329469710588455, 0.010839399881660938, 0.0005578729906119406, 101.0, 45.0]
Loaded "alibaba_cluster_trace"
